In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

In [ ]:
cd /home/jupyter/

In [ ]:
df_train = pd.read_parquet('intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip')
df_test = pd.read_parquet('intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip')

Build a XGBoost model to classify event segments

In [ ]:
X_train = df_train.loc[:,~df_train.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_train = df_train['veranst_segment']

X_test = df_test.loc[:,~df_test.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_test = df_test['veranst_segment']

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [ ]:
xgboost_classifier = xgboost.XGBClassifier(n_estimators=500, max_depth=7, use_label_encoder=False, objective='binary:logistic',eval_metric = 'error')

In [ ]:
# xgboost_classifier = xgboost.XGBClassifier()
xgboost_classifier.fit(X_train, y_train)

In [ ]:
# save the model to disk
xbg_filename = 'intellizenz-model-training/models/2022-10-06/xgb_classifier_model_with_inkasso_time_series.sav'
pickle.dump(xgboost_classifier, open(xbg_filename, 'wb'))

In [ ]:
y_pred = xgboost_classifier.predict(X_test)

Evaluation

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

xgboost_accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(xgboost_accuracy)